# Prepare dataset

In [ ]:
# download, decompress the data
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.1/balloon_dataset.zip 
!unzip balloon_dataset.zip
!rm -r __MACOSX

In [42]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
prefix_input = 'detectron2-input'
prefix_output = 'detectron2-ouput'

In [15]:
# now need to upload it to S3

import os
import boto3

s3_resource = boto3.resource("s3", region_name=region)
root_path = os.getcwd()
data_path = os.path.join(root_path, "balloon")

for path, subdirs, files in os.walk(data_path):
    directory_name = path.replace(root_path+"/","")
    for file in files:
        #print(os.path.join(root_path, directory_name, file))
        s3_resource.Bucket(bucket).upload_file(os.path.join(root_path, directory_name, file), directory_name+'/'+file)
        #print(f"file {file} has been uploaded to S3")

# Train model

Let's review the training script.

In [33]:
! pygmentize d2_script/train.py

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog


# packages neededs for custom dataset
import os
import numpy as np
import json
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog


def train():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ("balloon_train",)
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
    cfg.SOLVER.IMS_PE

In [47]:
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role

role = get_execution_role()

d2 = PyTorch(entry_point="train.py", source_dir="d2_script",
             train_instance_count=2,
             role=role,
             train_instance_type='ml.p3.2xlarge',
             framework_version="1.4.0")

#d2.set_hyperparameters(num_epochs = 1, num_classes = 2, )

d2.fit({'train':"s3://sagemaker-us-east-2-553020858742/balloon/train",
        'val':"s3://sagemaker-us-east-2-553020858742/balloon/val"})

2020-04-06 20:25:37 Starting - Starting the training job...
2020-04-06 20:25:39 Starting - Launching requested ML instances...
2020-04-06 20:26:36 Starting - Preparing the instances for training.........
2020-04-06 20:27:50 Downloading - Downloading input data...
2020-04-06 20:28:25 Training - Downloading the training image......
2020-04-06 20:29:38 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-06 20:29:35,157 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-06 20:29:35,181 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-04-06 20:29:39,591 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-04-06 20:29:39,616 

UnexpectedStatusException: Error for Training job pytorch-training-2020-04-06-20-25-34-343: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python train.py"
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-oke1q4r3
model_final_f10217.pkl: 0.00B [00:00, ?B/s]model_final_f10217.pkl:   0%|          | 8.19k/178M [00:00<4:03:38, 12.2kB/s]model_final_f10217.pkl:   0%|          | 57.3k/178M [00:00<2:53:31, 17.1kB/s]model_final_f10217.pkl:   0%|          | 262k/178M [00:00<2:01:58, 24.3kB/s] model_final_f10217.pkl:   1%|          | 1.17M/178M [00:01<1:25:06, 34.6kB/s]model_final_f10217.pkl:   2%|â         | 4.14M/178M [00:01<58:37, 49.4kB/s]  model_final_f10217.pkl:   4%|â         | 7.99M/178M [00:01<40:09, 70.5kB/s]model_final_f10217.pkl:   6%|â         | 11.3M/178M [00:01<27:36, 101kB/s] model_final_f10217.pkl:   8%|â         | 15.0M/178M [00:01<18:55, 143kB/s]model_final_f10217.pkl:  10%|â       